In [1]:
import pandas as pd
import numpy as np
import datetime

## Study population for 90-day post-discharge mortality prediction

In [3]:
icustays_MV = pd.read_csv('/data4/tangsp/FIDDLE/mimic3_experiments/data/processed/prep/icustays_MV.csv')

In [4]:
mimic3_path = '/data/mimic3/'
admission = pd.read_csv(mimic3_path+"ADMISSIONS.csv")
patients = pd.read_csv(mimic3_path+"PATIENTS.csv")

In [5]:
# already split data partitions at patient level
splits = icustays_MV[['SUBJECT_ID', 'partition']].drop_duplicates()

In [6]:
admissions_MV = admission[admission['HADM_ID'].isin(icustays_MV['HADM_ID'].unique())]

In [7]:
admissions_MV.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
13,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,2191-02-23 04:23:00,2191-02-23 07:25:00,BRADYCARDIA,0,1
17,38,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,VENTRAL HERNIA/SDA,0,1
25,459,357,101651,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2199-10-20 09:34:00,2199-10-20 13:25:00,PNEUMONIA,0,1
26,460,357,117876,2199-12-21 22:37:00,2200-01-18 11:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2199-12-21 20:38:00,2199-12-22 00:01:00,ALTERED MENTAL STATUS,0,1


In [8]:
admissions_MV.loc[admissions_MV['HOSPITAL_EXPIRE_FLAG'] == 1, 'DISCHARGE_LOCATION'].unique()

array(['DEAD/EXPIRED'], dtype=object)

In [9]:
examples = admissions_MV[['SUBJECT_ID','HADM_ID', 'HOSPITAL_EXPIRE_FLAG', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME']]
print('Admissions MV, N =', len(examples))

# Remove deaths at discharge
deaths = examples[examples['HOSPITAL_EXPIRE_FLAG'] == 1]
examples = examples[examples['HOSPITAL_EXPIRE_FLAG'] != 1]
print('Deaths at discharge: ', len(deaths))
print('Survival at discharge: ', len(examples))

Admissions MV, N = 22046
Deaths at discharge:  2323
Survival at discharge:  19723


In [10]:
examples.head()

,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG,ADMITTIME,DISCHTIME,DEATHTIME
2,23,124321,0,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN
13,34,144319,0,2191-02-23 05:23:00,2191-02-25 20:20:00,NaN
17,36,165660,0,2134-05-10 11:30:00,2134-05-20 13:16:00,NaN
25,357,101651,0,2199-10-20 12:05:00,2199-10-23 17:30:00,NaN
26,357,117876,0,2199-12-21 22:37:00,2200-01-18 11:45:00,NaN


In [11]:
examples_splits = examples.merge(splits)
examples_splits.to_csv('data/population.csv', index=False)

## Get labels

### Hospital stay length

In [11]:
examples.iloc[:, 3:7] = examples.iloc[:, 3:7].apply(pd.to_datetime)

In [12]:
los = (examples['DISCHTIME'] - examples['ADMITTIME'])

In [13]:
los.max()

Timedelta('169 days 04:30:00')

### 90-day mortality post-discharge

In [14]:
df = examples.merge(patients[['SUBJECT_ID', 'DOD']])
df['DOD'] = df['DOD'].apply(pd.to_datetime)

In [15]:
df.head()

,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG,ADMITTIME,DISCHTIME,DEATHTIME,DOD
0,23,124321,0,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,NaT
1,34,144319,0,2191-02-23 05:23:00,2191-02-25 20:20:00,NaT,2192-01-30
2,36,165660,0,2134-05-10 11:30:00,2134-05-20 13:16:00,NaT,NaT
3,357,101651,0,2199-10-20 12:05:00,2199-10-23 17:30:00,NaT,2201-08-02
4,357,117876,0,2199-12-21 22:37:00,2200-01-18 11:45:00,NaT,2201-08-02


In [16]:
df['mortality_day'] = pd.Series(np.where(df['DOD'].isnull(), 0, df['DOD'] - df['DISCHTIME'])).dt.days
df['label'] = np.where((df['mortality_day']>0) & (df['mortality_day']<=90), 1, 0)

In [17]:
df.head(6)

,SUBJECT_ID,HADM_ID,HOSPITAL_EXPIRE_FLAG,ADMITTIME,DISCHTIME,DEATHTIME,DOD,mortality_day,label
0,23,124321,0,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,NaT,0,0
1,34,144319,0,2191-02-23 05:23:00,2191-02-25 20:20:00,NaT,2192-01-30,338,0
2,36,165660,0,2134-05-10 11:30:00,2134-05-20 13:16:00,NaT,NaT,0,0
3,357,101651,0,2199-10-20 12:05:00,2199-10-23 17:30:00,NaT,2201-08-02,647,0
4,357,117876,0,2199-12-21 22:37:00,2200-01-18 11:45:00,NaT,2201-08-02,560,0
5,85,112077,0,2167-07-25 18:49:00,2167-07-30 15:24:00,NaT,2167-09-12,43,1


In [18]:
len(df.SUBJECT_ID.unique())

15870

In [19]:
df.to_csv('data/label.csv', index=False)

### proportion of examples that died within 90 days post-discharge 

In [20]:
np.mean(df.label)

0.09445824671703087

In [21]:
sum(df.label)

1863

In [22]:
df.shape[0]-sum(df.label)

17860

## Get data

### ICD codes at discharge

In [12]:
ICD = pd.read_csv(mimic3_path+"DIAGNOSES_ICD.csv")

In [13]:
df_ICD = ICD[['HADM_ID', 'ICD9_CODE']].rename(columns={'HADM_ID': 'ID', 'ICD9_CODE': 'variable_value'}).copy()

In [14]:
df_ICD = df_ICD.loc[df_ICD['ID'].isin(examples['HADM_ID'])]

In [15]:
df_ICD['t'] = np.nan
df_ICD['variable_name'] = 'ICD9_CODE'
df_ICD = df_ICD[['ID', 't', 'variable_name', 'variable_value']]

In [16]:
df_ICD = df_ICD[df_ICD['ID'].isin(examples['HADM_ID'])]

In [17]:
df_ICD

,ID,t,variable_name,variable_value
0,172335,NaN,ICD9_CODE,40301
1,172335,NaN,ICD9_CODE,486
2,172335,NaN,ICD9_CODE,58281
3,172335,NaN,ICD9_CODE,5855
4,172335,NaN,ICD9_CODE,4254
...,...,...,...,...
651042,188195,NaN,ICD9_CODE,20280
651043,188195,NaN,ICD9_CODE,V5869
651044,188195,NaN,ICD9_CODE,V1279
651045,188195,NaN,ICD9_CODE,5275


In [19]:
df_ICD.dropna(subset=['variable_value']).to_csv('./data/icd_data.csv', index=False)

### Clinical data, remap ID and t

In [28]:
import pickle
import copy
from helper import *

In [29]:
# Calculate time offset, in hours
icustays_admissions = icustays_MV[['HADM_ID', 'ICUSTAY_ID', 'INTIME']].merge(admissions_MV[['HADM_ID', 'ADMITTIME']])
icustays_admissions[['INTIME', 'ADMITTIME']] = icustays_admissions[['INTIME', 'ADMITTIME']].apply(pd.to_datetime)

In [30]:
icustays_admissions['t_offset'] = (icustays_admissions['INTIME'] - icustays_admissions['ADMITTIME']).dt.total_seconds() / 3600.

In [31]:
offset_mappings = icustays_admissions[['HADM_ID', 'ICUSTAY_ID', 't_offset']]

In [32]:
offset_mappings

,HADM_ID,ICUSTAY_ID,t_offset
0,152234,200001,171.036667
1,192256,200010,0.021944
2,121562,200011,5.206667
3,117458,200016,4.988889
4,109307,200021,0.020000
...,...,...,...
23615,110990,299950,0.024722
23616,108958,299956,10.315000
23617,129555,299957,115.438056
23618,146497,299962,0.018333


In [33]:
with open('/data4/tangsp/FIDDLE/mimic3_experiments/data/processed/formatted/all_data.stacked.p', 'rb') as f:
    data = pickle.load(f)

In [34]:
data.keys()

dict_keys(['TIME_INVARIANT', 'LABEVENTS', 'MICROBIOLOGYEVENTS', 'DATETIMEEVENTS', 'OUTPUTEVENTS', 'CHARTEVENTS', 'INPUTEVENTS_MV', 'PROCEDUREEVENTS_MV'])

In [35]:
data_old = copy.deepcopy(data)

In [36]:
# remap ID and add time offset
for name, df in tqdm(data.items()):
    if name == 'TIME_INVARIANT':
        continue
    
    df = df.merge(offset_mappings, left_on='ID', right_on='ICUSTAY_ID')
    df_new = df.copy()
    if 't' in df_new.columns:
        df_new['t'] = df['t'] + df['t_offset']
    elif 't_start' in df_new.columns:
        df_new['t_start'] = df['t_start'] + df['t_offset']
        df_new['t_end'] = df['t_end'] + df['t_offset']

    df_new['ID'] = df['HADM_ID']
    if 't' in df_new.columns:
        df_new = df_new[['ID', 't', 'variable_name', 'variable_value']]
    elif 't_start' in df_new.columns:
        df_new = df_new[['ID', 't_start', 't_end', 'variable_name', 'variable_value']]
    data[name] = df_new

100%|██████████| 8/8 [13:07<00:00, 98.50s/it] 


In [37]:
T = 180*24. # max hospLOS = 180 days
dt = 1.0
data = filter_prediction_time(data, T) 


--------------------------------------------------------------------------------
Filter by prediction time T=4320.0
--------------------------------------------------------------------------------


100%|██████████| 8/8 [03:19<00:00, 24.92s/it] 

Done!


In [38]:
# Resample continuous, resolve duplicates (discrete & continuous)
data = resolve_duplicates_discrete(data)
data = resample_continuous_events(data, T, dt)
# data = resolve_duplicates_continuous(data)



--------------------------------------------------------------------------------
Resolve duplicated event records (discrete)
--------------------------------------------------------------------------------
*** CHARTEVENTS
    getting dups and ~dups


[Parallel(n_jobs=64)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=64)]: Done  17 tasks      | elapsed:  1.7min
[Parallel(n_jobs=64)]: Done  34 tasks      | elapsed:  1.8min
[Parallel(n_jobs=64)]: Done  53 tasks      | elapsed:  1.9min
[Parallel(n_jobs=64)]: Done  72 tasks      | elapsed:  2.0min
[Parallel(n_jobs=64)]: Done 105 out of 210 | elapsed:  2.0min remaining:  2.0min
[Parallel(n_jobs=64)]: Done 127 out of 210 | elapsed:  2.0min remaining:  1.3min
[Parallel(n_jobs=64)]: Done 149 out of 210 | elapsed:  2.1min remaining:   50.8s
[Parallel(n_jobs=64)]: Done 171 out of 210 | elapsed:  2.1min remaining:   28.9s
[Parallel(n_jobs=64)]: Done 193 out of 210 | elapsed:  2.2min remaining:   11.5s
[Parallel(n_jobs=64)]: Done 210 out of 210 | elapsed:  8.3min finished


    concatenating results
*** LABEVENTS
Verifying no more duplicates...
MICROBIOLOGYEVENTS contains duplicate records

--------------------------------------------------------------------------------
Resample continuous events, T=4320.0, dt=1.0
--------------------------------------------------------------------------------


  0%|          | 26/255828 [00:00<1:45:47, 40.30it/s]

PROCEDUREEVENTS_MV


  0%|          | 0/8678376 [00:00<?, ?it/s]

INPUTEVENTS_MV


 11%|█▏        | 979772/8678376 [52:24<6:51:31, 311.79it/s] 

KeyboardInterrupt: 

In [41]:
data.keys()

dict_keys(['CHARTEVENTS', 'DATETIMEEVENTS', 'INPUTEVENTS_MV', 'LABEVENTS', 'MICROBIOLOGYEVENTS', 'OUTPUTEVENTS', 'PROCEDUREEVENTS_MV', 'TIME_INVARIANT'])

In [42]:
data_ = { name: data[name] for name in [
    'CHARTEVENTS', 'DATETIMEEVENTS', 'LABEVENTS', 'MICROBIOLOGYEVENTS', 'OUTPUTEVENTS', 'TIME_INVARIANT']
}

In [43]:
# Combine all DataFrames into one
df_data = pd.concat(data_, axis='index', ignore_index=True)
df_data = df_data.sort_values(by=['ID', 't', 'variable_name', 'variable_value'], na_position='first')

In [44]:
df_data.to_pickle('data/clinical.p')